In [ ]:
import matplotlib.pyplot
import numpy

In [ ]:
l = 9.9 # sm

In [ ]:
class Experiment:
    def __init__ (self, T, glass_ds, metal_ds, times_glass, times_steel):
        self.T = T
        self.glass_ds = glass_ds
        self.metal_ds = metal_ds
        self.times_glass = times_glass
        self.times_steel = times_steel

In [ ]:
Ts = [25.40, 35]
glass_ds = [2.1, 2.05,
            2.15, ] # mm
metal_ds = [0.7, 0.9] # mm
times_T_0 = [    # glass
            [24.45, 24.77],
            [24.78, 24.26],
                # steel
            [37.21, 37.21],
            [24.91, 24.93]]

Experiment1 = Experiment (25.40,
                          [2.1, 2.05, 2.15],
                          [0.7, 0.9],
                          [[24.45, 24.47], [24.78, 24.26]],
                          [[37.21, 37.21], [24.91, 24.93]])

In [ ]:
Experiment2 = Experiment (35.83,
                          [2.15, 2.15],
                          [0.8, 0.75],
                          [[11.55, 11.59], [11.11, 11.47]],
                          [[11.46, 11.49], [16.22, 16.20]])

In [ ]:
Experiment3 = Experiment (
    40.83,
    [2.1, 2.05],
    [0.9, 0.6],
    [[9.93, 10.09], [9.30, 9.23]],
    [[10.47, 10.93], [13.65, 13.63]]
)

In [ ]:
Experiment4 = Experiment (
    50.85,
    [2.15, 2.10],
    [0.75, 0.7],
    [[5, 5.34], [4.62, 4.84]],
    [[7.72, 7.94], [7.15, 7.49]]
)

In [ ]:
Experiment5 = Experiment (
    60.63,
    [2.2, 2],
    [0.7, 0.75],
    [[2.59, 3.18], [2.49, 2.78]],   # температура уже 60
    [[4.16, 4.49], [4.93, 5.62]]
)
# последний шарик долго не получался изза кривых рук и шуток судьбы